#Project 1 - 
##Developing an Equal-Weight S&P 500 Index Fund
The S&P 500 is the world's most popular stock market index that has more than US$250 billion of assets under management.

**Disclaimer: This project is only for the purpose of Algorithmic Demonstration and is NOT A FINANCIAL ADVICE OR CONCLUSION IN ANY FORM.**

###Aim
Determining the Number of shares to invest in each of the stocks listed in the index based on the Price of the Stock and the Value of the Portfolio to get an Equal-Weight version of the Index Fund.

*   Reference Project = https://github.com/nickmccullum/algorithmic-trading-python
*   Video Link = https://www.youtube.com/watch?v=xfzGZB4HhEE&t=1639s




In [1]:
#Importing the Libraries
import numpy as np
import pandas as pd
import requests
from scipy import stats
import math

In [2]:
#Uploading the Dataset
from google.colab import files
uploaded = files.upload()

Saving sp_500_stocks.csv to sp_500_stocks.csv


In [3]:
#Reading the Data Set
data_stock = pd.read_csv("sp_500_stocks.csv")
data_stock.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


#Importing Stock related Data using API Calls
The IEX Cloud is used to import the data related to the stocks in the Index. The data is imported by the means of an API Call 
#Creating a S&P 500 Stock Dataframe 
Creating a Pandas Dataframe to store all the Stock Tickers and the information encompassing the Stock Price and Market Capitalization associated with each Ticker.

In [ ]:
cols = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
recommended_trades = pd.DataFrame(columns = cols)
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
for tick in data_stock['Ticker']:
    url_API = f'https://sandbox.iexapis.com/stable/stock/{tick}/quote?token={IEX_CLOUD_API_TOKEN}'
    stockdata_API = requests.get(url_API).json()
    recommended_trades = recommended_trades.append(
                                        pd.Series([tick, 
                                                   stockdata_API['latestPrice'], 
                                                   stockdata_API['marketCap'], 
                                                   'N/A'], 
                                                  index = cols), 
                                        ignore_index = True)
recommended_trades

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,161.03,47075583217,N/A
1,AAL,18.94,12087306343,N/A
2,AAP,230.74,14403508548,N/A
3,AAPL,177.69,2915980499752,N/A
4,ABBV,134.00,233456105928,N/A
...,...,...,...,...
500,YUM,138.77,41007661847,N/A
501,ZBH,128.26,26504795469,N/A
502,ZBRA,585.39,31303974531,N/A
503,ZION,62.79,10044369596,N/A


#Estimating the Number of Shares
Based on the Portfolio Size we then estimate the number of shares to be bought per stock in the index. The Number of shares to be bought are weighted such that less shares are bought of stocks with a higher price and more shares are bought for stocks with a lower price.



In [ ]:
portfolio_size_case1 = input("Enter the value of your portfolio in Dollars:")
try:
    val = float(portfolio_size_case1)
except ValueError:
    print("Invalid Portfolio Size:")
    portfolio_size_case1 = input("Enter the value of your portfolio:")

Enter the value of your portfolio in Dollars:1000000


In [ ]:
position_size_case1 = float(portfolio_size_case1) / len(recommended_trades.index)
for i in range(0, len(recommended_trades['Ticker'])-1):
    recommended_trades.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size_case1 / recommended_trades['Price'][i])
recommended_trades

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,A,161.03,47075583217,12
1,AAL,18.94,12087306343,104
2,AAP,230.74,14403508548,8
3,AAPL,177.69,2915980499752,11
4,ABBV,134.00,233456105928,14
...,...,...,...,...
500,YUM,138.77,41007661847,14
501,ZBH,128.26,26504795469,15
502,ZBRA,585.39,31303974531,3
503,ZION,62.79,10044369596,31


#Exporting the Data
Exporting the Final Portfolio Data Frame in the form an Excel Report  

In [ ]:
recommended_trades.to_excel("Recommended_Trades.xlsx")
print('DataFrame is written to Excel File successfully.')
from google.colab import files
files.download('Recommended_Trades.xlsx')

DataFrame is written to Excel File successfully.


#Project 2
##Investment Algorithm Based on a Momentum Strategy
**Momentum investing is a trading strategy involving buying securities that are rising and sell them when they look to have peaked. The goal is to find the investment opportunities based on short term volatality trends.**

In reality, severat investment firms differentiate between "high quality" and "low quality" momentum stocks:

* **High-quality momentum** stocks show consistent trends over long periods of time
* **Low-quality momentum** stocks show sudden surges even after stagnated momentum

In this part, use the concept of percentiles of 1-month price returns, 3-month price returns
6-month price returns & 1-year price returns to identify high-quality momentum stocks. 

In [ ]:
#Dividing the Data into chunks to improve iterative computational speeds
# Code used from: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
tick_group = list(chunks(data_stock['Ticker'], 100))
tick_string = []
for i in range(0, len(tick_group)):
    tick_string.append(','.join(tick_group[i]))

In [ ]:
cols_2 = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

momentum_strat_data = pd.DataFrame(columns = cols_2)
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
for j in tick_string:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={j}&token={IEX_CLOUD_API_TOKEN}'
    momstrat_data = requests.get(batch_api_call_url).json()
    for tick_1 in j.split(','):
        momentum_strat_data = momentum_strat_data.append(
                                        pd.Series([tick_1, 
                                                   momstrat_data[tick_1]['quote']['latestPrice'],
                                                   'N/A',
                                                   momstrat_data[tick_1]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   momstrat_data[tick_1]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   momstrat_data[tick_1]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   momstrat_data[tick_1]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = cols_2), 
                                        ignore_index = True)
        
momentum_strat_data

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.900,N/A,0.329444,N/A,0.0565281,N/A,-0.105447,N/A,-0.06312,N/A,N/A
1,AAL,18.450,N/A,0.127427,N/A,-0.197564,N/A,-0.0840723,N/A,-0.0628443,N/A,N/A
2,AAP,239.700,N/A,0.468975,N/A,0.171179,N/A,0.126058,N/A,0.00238819,N/A,N/A
3,AAPL,174.080,N/A,0.369927,N/A,0.326874,N/A,0.213373,N/A,0.0791597,N/A,N/A
4,ABBV,133.900,N/A,0.328091,N/A,0.163306,N/A,0.23281,N/A,0.121595,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.631,N/A,0.257139,N/A,0.151619,N/A,0.0748186,N/A,0.0554749,N/A,N/A
501,ZBH,123.160,N/A,-0.176887,N/A,-0.260201,N/A,-0.14555,N/A,-0.0516616,N/A,N/A
502,ZBRA,591.190,N/A,0.563752,N/A,0.15651,N/A,0.070126,N/A,-0.0278692,N/A,N/A
503,ZION,64.450,N/A,0.492534,N/A,0.189699,N/A,0.125198,N/A,-0.0577545,N/A,N/A


#Calculating Momentum Percentiles


In [ ]:
momentum_strat_data = momentum_strat_data.dropna()
momentum_strat_data.isna().sum()

Ticker                           0
Price                            0
Number of Shares to Buy          0
One-Year Price Return            0
One-Year Return Percentile       0
Six-Month Price Return           0
Six-Month Return Percentile      0
Three-Month Price Return         0
Three-Month Return Percentile    0
One-Month Price Return           0
One-Month Return Percentile      0
HQM Score                        0
dtype: int64

In [ ]:
timeperiod = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in momentum_strat_data.index:
    for t in timeperiod:
        momentum_strat_data.loc[row, f'{t} Return Percentile'] = stats.percentileofscore(momentum_strat_data[f'{t} Price Return'], momentum_strat_data.loc[row, f'{t} Price Return'])
       
momentum_strat_data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.900,N/A,0.329444,59.481,0.0565281,47.9042,-0.105447,7.78443,-0.06312,13.3733,0.321357
1,AAL,18.450,N/A,0.127427,30.3393,-0.197564,5.38922,-0.0840723,10.3792,-0.0628443,13.7725,0.235529
2,AAP,239.700,N/A,0.468975,78.6427,0.171179,74.8503,0.126058,74.4511,0.00238819,54.0918,0.392049
3,AAPL,174.080,N/A,0.369927,65.6687,0.326874,94.4112,0.213373,91.4172,0.0791597,91.8164,0.508608
4,ABBV,133.900,N/A,0.328091,59.2814,0.163306,72.6547,0.23281,93.014,0.121595,97.006,0.567864
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.631,N/A,0.257139,50.8982,0.151619,70.0599,0.0748186,55.2894,0.0554749,84.2315,0.500657
501,ZBH,123.160,N/A,-0.176887,3.39321,-0.260201,2.99401,-0.14555,2.79441,-0.0516616,20.3593,0.4998
502,ZBRA,591.190,N/A,0.563752,86.4271,0.15651,71.2575,0.070126,53.6926,-0.0278692,32.9341,0.500022
503,ZION,64.450,N/A,0.492534,80.8383,0.189699,80.6387,0.125198,74.0519,-0.0577545,16.7665,0.500283


#Calculating the aggregated High Quality Momentum Score
We Calculate the Final High Quality Momentum Score for each stock by taking the Average (Mean) of the One Year, Six-Months, Three-Months and One-Month percentiles values of each of the stocks.  


In [ ]:
from statistics import mean
momentum_percentiles = []
for row in momentum_strat_data.index:
    for t in timeperiod:
        momentum_percentiles.append(momentum_strat_data.loc[row, f'{t} Return Percentile'])
    momentum_strat_data.loc[row, 'HQM Score'] = mean(momentum_percentiles)

momentum_strat_data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.900,N/A,0.329444,59.481,0.0565281,47.9042,-0.105447,7.78443,-0.06312,13.3733,32.1357
1,AAL,18.450,N/A,0.127427,30.3393,-0.197564,5.38922,-0.0840723,10.3792,-0.0628443,13.7725,23.5529
2,AAP,239.700,N/A,0.468975,78.6427,0.171179,74.8503,0.126058,74.4511,0.00238819,54.0918,39.2049
3,AAPL,174.080,N/A,0.369927,65.6687,0.326874,94.4112,0.213373,91.4172,0.0791597,91.8164,50.8608
4,ABBV,133.900,N/A,0.328091,59.2814,0.163306,72.6547,0.23281,93.014,0.121595,97.006,56.7864
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,139.631,N/A,0.257139,50.8982,0.151619,70.0599,0.0748186,55.2894,0.0554749,84.2315,50.0657
501,ZBH,123.160,N/A,-0.176887,3.39321,-0.260201,2.99401,-0.14555,2.79441,-0.0516616,20.3593,49.98
502,ZBRA,591.190,N/A,0.563752,86.4271,0.15651,71.2575,0.070126,53.6926,-0.0278692,32.9341,50.0022
503,ZION,64.450,N/A,0.492534,80.8383,0.189699,80.6387,0.125198,74.0519,-0.0577545,16.7665,50.0283


#Selecting the 50 Highest Momentum Stocks


In [ ]:
momentum_strat_data.sort_values(by = 'HQM Score', ascending = False)
momentum_strat_data = momentum_strat_data[:51]
momentum_strat_data

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.900,N/A,0.329444,59.481,0.0565281,47.9042,-0.105447,7.78443,-0.06312,13.3733,32.1357
1,AAL,18.450,N/A,0.127427,30.3393,-0.197564,5.38922,-0.0840723,10.3792,-0.0628443,13.7725,23.5529
2,AAP,239.700,N/A,0.468975,78.6427,0.171179,74.8503,0.126058,74.4511,0.00238819,54.0918,39.2049
3,AAPL,174.080,N/A,0.369927,65.6687,0.326874,94.4112,0.213373,91.4172,0.0791597,91.8164,50.8608
4,ABBV,133.900,N/A,0.328091,59.2814,0.163306,72.6547,0.23281,93.014,0.121595,97.006,56.7864
5,ABC,132.300,N/A,0.338711,61.477,0.103132,60.0798,0.0559484,49.7006,0.0591438,85.4291,58.0173
6,ABMD,339.140,N/A,0.123558,29.5409,0.0223839,38.9222,-0.0382658,17.5649,-0.023429,36.3273,54.0989
7,ABT,143.860,N/A,0.279612,53.493,0.232891,86.8263,0.0890894,60.2794,0.0691266,88.8224,56.381
8,ACN,410.637,N/A,0.547431,84.8303,0.387097,97.2056,0.191428,89.8204,0.0654413,87.8244,60.1076
9,ADBE,578.960,N/A,0.120561,28.7425,-0.0178671,29.5409,-0.137348,3.19361,-0.190386,1.1976,55.6637


#Estimating the Number of Shares
Based on the Portfolio Size we then estimate the number of shares to be bought per stock in the index. The Number of shares to be bought are weighted such that less shares are bought of stocks with a higher price and more shares are bought for stocks with a lower price.

In [ ]:
portfolio_size_case2 = input("Enter the value of your portfolio in Dollars:")
try:
    val = float(portfolio_size_case2)
except ValueError:
    print("Invalid Portfolio Size:")
    portfolio_size_case2 = input("Enter the value of your portfolio:")

Enter the value of your portfolio in Dollars:10000


In [ ]:
position_size_case2 = float(portfolio_size_case2) / len(momentum_strat_data.index)
for i in range(0, len(momentum_strat_data['Ticker'])-1):
    momentum_strat_data.loc[i, 'Number of Shares to Buy'] = math.floor(position_size_case2 / momentum_strat_data['Price'][i])
momentum_strat_data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,157.900,1,0.329444,59.481,0.0565281,47.9042,-0.105447,7.78443,-0.06312,13.3733,32.1357
1,AAL,18.450,10,0.127427,30.3393,-0.197564,5.38922,-0.0840723,10.3792,-0.0628443,13.7725,23.5529
2,AAP,239.700,0,0.468975,78.6427,0.171179,74.8503,0.126058,74.4511,0.00238819,54.0918,39.2049
3,AAPL,174.080,1,0.369927,65.6687,0.326874,94.4112,0.213373,91.4172,0.0791597,91.8164,50.8608
4,ABBV,133.900,1,0.328091,59.2814,0.163306,72.6547,0.23281,93.014,0.121595,97.006,56.7864
5,ABC,132.300,1,0.338711,61.477,0.103132,60.0798,0.0559484,49.7006,0.0591438,85.4291,58.0173
6,ABMD,339.140,0,0.123558,29.5409,0.0223839,38.9222,-0.0382658,17.5649,-0.023429,36.3273,54.0989
7,ABT,143.860,1,0.279612,53.493,0.232891,86.8263,0.0890894,60.2794,0.0691266,88.8224,56.381
8,ACN,410.637,0,0.547431,84.8303,0.387097,97.2056,0.191428,89.8204,0.0654413,87.8244,60.1076
9,ADBE,578.960,0,0.120561,28.7425,-0.0178671,29.5409,-0.137348,3.19361,-0.190386,1.1976,55.6637


#Exporting the Data
Exporting the Final Portfolio Data Frame in the form an Excel Report  

In [ ]:
momentum_strat_data.to_excel("Momentum_Investing_Watchlist.xlsx")
print('DataFrame is written to Excel File successfully.')
from google.colab import files
files.download('Momentum_Investing_Watchlist.xlsx')

#Project 3
##Implementing a Value Strategy Investment Algorithm
Value investing is an investment strategy that involves picking stocks that appear to be trading for less than their intrinsic or book value. Stocks that investors think are understimated are invested into as market influences (Market Overreacts to such stocks) cause prominent fluctuations in their values. Buy/Sell opportunities can be realised from such fluctuations.  

Several KPIs are considered to identify such underestimated stocks.But relevant KPIs are specific to the kind of stock we deal with. Hence, for a complete index selecting KPIs becomes a little tricky.

In order to address to the relevancy concerns of the KPIs, we consider the aggregate indications of the following metrics (Lowest Average of the Percentiles of the metrics for each stock):
* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value to EBITDA
* Enterprise Value to Gross Profit 

In [25]:
# Function sourced from: https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]           
new_tick_group= list(chunks(data_stock['Ticker'], 100))
new_tick_string = []
for i in range(0, len(new_tick_group)):
    new_tick_string.append(','.join(new_tick_group[i]))

In [26]:
val_cols = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'Enterprise Value to EBITDA',
    'EV/EBITDA Percentile',
    'Enterprise Value to Gross Profit',
    'EV/GP Percentile',
    'RV Score'
]

val_strat_data = pd.DataFrame(columns = val_cols)
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
for a in new_tick_string:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={a}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data_val_strat = requests.get(batch_api_call_url).json()
    for b in a.split(','):
        enterprise_value = data_val_strat[b]['advanced-stats']['enterpriseValue']
        ebitda = data_val_strat[b]['advanced-stats']['EBITDA']
        gross_profit = data_val_strat[b]['advanced-stats']['grossProfit']
        
        try:
            ev_ebitda_ratio = enterprise_value/ebitda
        except TypeError:
            ev_ebitda_ratio = np.NaN
        
        try:
            ev_gross_profit_ratio = enterprise_value/gross_profit
        except TypeError:
            ev_gross_profit_ratio = np.NaN
            
        val_strat_data = val_strat_data.append(
            pd.Series([
                b,
                data_val_strat[b]['quote']['latestPrice'],
                'N/A',
                data_val_strat[b]['quote']['peRatio'],
                'N/A',
                data_val_strat[b]['advanced-stats']['priceToBook'],
                'N/A',
                data_val_strat[b]['advanced-stats']['priceToSales'],
                'N/A',
                ev_ebitda_ratio,
                'N/A',
                ev_gross_profit_ratio,
                'N/A',
                'N/A'
        ],
        index = val_cols),
            ignore_index = True
        )

val_strat_data

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,Enterprise Value to EBITDA,EV/EBITDA Percentile,Enterprise Value to Gross Profit,EV/GP Percentile,RV Score
0,A,157.49,N/A,50.55,N/A,9.7,N/A,7.65,N/A,29.839952,N/A,14.012935,N/A,N/A
1,AAL,18.41,N/A,-3.71,N/A,-1.59,N/A,0.4877,N/A,-7.310569,N/A,1.455452,N/A,N/A
2,AAP,234.62,N/A,23.94,N/A,4.64,N/A,1.36,N/A,13.159008,N/A,2.993636,N/A,N/A
3,AAPL,180.65,N/A,15.92,N/A,22.8,N/A,4.03,N/A,12.273226,N/A,9.517170,N/A,N/A
4,ABBV,136.62,N/A,32.63,N/A,17.32,N/A,4.26,N/A,11.428457,N/A,8.047209,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.83,N/A,27.11,N/A,-5.11,N/A,6.09,N/A,21.462079,N/A,10.089391,N/A,N/A
501,ZBH,129.67,N/A,33.2,N/A,2.1,N/A,3.31,N/A,17.656184,N/A,5.598055,N/A,N/A
502,ZBRA,595.53,N/A,38.72,N/A,11.13,N/A,5.83,N/A,29.281708,N/A,12.314034,N/A,N/A
503,ZION,63.28,N/A,6.13,N/A,1.34,N/A,2.8,N/A,4.485758,N/A,2.511953,N/A,N/A


In [27]:
#removing Null Values
#rv_dataframe[rv_dataframe.isnull().any(axis=1)]
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'Enterprise Value to EBITDA','Enterprise Value to Gross Profit']:
    val_strat_data[column].fillna(val_strat_data[column].mean(), inplace = True)
val_strat_data[val_strat_data.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,Enterprise Value to EBITDA,EV/EBITDA Percentile,Enterprise Value to Gross Profit,EV/GP Percentile,RV Score


#Calculating Value Percentiles


In [30]:
KPI = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'Enterprise Value to EBITDA':'EV/EBITDA Percentile',
            'Enterprise Value to Gross Profit':'EV/GP Percentile'
}

for row in val_strat_data.index:
    for metric in KPI.keys():
        val_strat_data.loc[row, KPI[metric]] = stats.percentileofscore(val_strat_data[metric], val_strat_data.loc[row, metric])
 
val_strat_data

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,Enterprise Value to EBITDA,EV/EBITDA Percentile,Enterprise Value to Gross Profit,EV/GP Percentile,RV Score
0,A,157.49,N/A,50.55,83.1683,9.70,80.5941,7.6500,82.3762,29.839952,83.3663,14.012935,83.1683,0.825347
1,AAL,18.41,N/A,-3.71,5.34653,-1.59,5.54455,0.4877,4.9505,-7.310569,2.37624,1.455452,6.33663,0.0491089
2,AAP,234.62,N/A,23.94,48.9109,4.64,60,1.3600,20.297,13.159008,43.7624,2.993636,16.4356,0.378812
3,AAPL,180.65,N/A,15.92,30.297,22.80,93.8614,4.0300,59.703,12.273226,40,9.517170,66.9307,0.581584
4,ABBV,136.62,N/A,32.63,65.3465,17.32,91.2871,4.2600,61.9802,11.428457,36.2376,8.047209,56.6337,0.62297
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.83,N/A,27.11,56.4356,-5.11,5.14851,6.0900,75.8416,21.462079,70.6931,10.089391,69.3069,0.554851
501,ZBH,129.67,N/A,33.20,66.3366,2.10,30.198,3.3100,51.4851,17.656184,60.198,5.598055,37.6238,0.491683
502,ZBRA,595.53,N/A,38.72,74.2574,11.13,83.7624,5.8300,74.6535,29.281708,82.5743,12.314034,77.6238,0.785743
503,ZION,63.28,N/A,6.13,9.50495,1.34,14.9505,2.8000,44.9505,4.485758,6.13861,2.511953,12.8713,0.176832


#Calculating the Robust Value Score
We calculate the Robust Value (RV) Score by finding the mean of the percentile scores of the KPIs considered.



In [33]:
from statistics import mean

for row in val_strat_data.index:
    percentiles_list = []
    for metric in KPI.keys():
       percentiles_list.append(val_strat_data.loc[row, KPI[metric]])
    val_strat_data.loc[row, 'Robust Value Score'] = mean(percentiles_list)
    
val_strat_data

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,Enterprise Value to EBITDA,EV/EBITDA Percentile,Enterprise Value to Gross Profit,EV/GP Percentile,RV Score,Robust Value Score
0,A,157.49,N/A,50.55,83.1683,9.70,80.5941,7.6500,82.3762,29.839952,83.3663,14.012935,83.1683,82.5347,82.534653
1,AAL,18.41,N/A,-3.71,5.34653,-1.59,5.54455,0.4877,4.9505,-7.310569,2.37624,1.455452,6.33663,4.91089,4.910891
2,AAP,234.62,N/A,23.94,48.9109,4.64,60,1.3600,20.297,13.159008,43.7624,2.993636,16.4356,37.8812,37.881188
3,AAPL,180.65,N/A,15.92,30.297,22.80,93.8614,4.0300,59.703,12.273226,40,9.517170,66.9307,58.1584,58.158416
4,ABBV,136.62,N/A,32.63,65.3465,17.32,91.2871,4.2600,61.9802,11.428457,36.2376,8.047209,56.6337,62.297,62.297030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,137.83,N/A,27.11,56.4356,-5.11,5.14851,6.0900,75.8416,21.462079,70.6931,10.089391,69.3069,55.4851,55.485149
501,ZBH,129.67,N/A,33.20,66.3366,2.10,30.198,3.3100,51.4851,17.656184,60.198,5.598055,37.6238,49.1683,49.168317
502,ZBRA,595.53,N/A,38.72,74.2574,11.13,83.7624,5.8300,74.6535,29.281708,82.5743,12.314034,77.6238,78.5743,78.574257
503,ZION,63.28,N/A,6.13,9.50495,1.34,14.9505,2.8000,44.9505,4.485758,6.13861,2.511953,12.8713,17.6832,17.683168


#Selecting the 50 Best Value Stocks¶
As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [34]:
val_strat_data.sort_values(by = 'Robust Value Score', inplace = True)
val_strat_data = val_strat_data[:50]
val_strat_data.reset_index(drop = True, inplace = True)
val_strat_data

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,Enterprise Value to EBITDA,EV/EBITDA Percentile,Enterprise Value to Gross Profit,EV/GP Percentile,RV Score,Robust Value Score
0,UNM,24.45,N/A,6.23,9.70297,0.4428,5.74257,0.3755,3.16832,2.887049,3.16832,0.352408,0.792079,4.51485,4.514851
1,AAL,18.41,N/A,-3.71,5.34653,-1.5900,5.54455,0.4877,4.9505,-7.310569,2.37624,1.455452,6.33663,4.91089,4.910891
2,HPQ,37.58,N/A,3.60,7.0297,-12.6600,3.56436,0.3295,1.78218,3.908813,4.75248,1.680340,8.51485,5.12871,5.128713
3,PRU,111.47,N/A,5.88,9.0099,0.6595,6.53465,0.5712,5.54455,4.461466,5.94059,0.589168,1.18812,5.64356,5.643564
4,BEN,33.25,N/A,4.60,7.72277,0.7531,7.72277,0.9956,12.6733,3.003482,3.56436,0.817845,1.9802,6.73267,6.732673
5,BA,210.53,N/A,-14.54,4.15842,-8.2300,4.75248,1.9000,30.7921,-32.481961,0.594059,-1320.185523,0.19802,8.09901,8.099010
6,KSS,49.76,N/A,7.95,12.6733,1.4100,16.8317,0.3721,2.9703,3.656899,3.9604,1.159481,4.9505,8.27723,8.277228
7,AIG,56.10,N/A,8.91,13.8614,0.7277,7.32673,0.9860,12.0792,4.387631,5.74257,0.979202,3.16832,8.43564,8.435644
8,VIAC,29.55,N/A,5.88,9.0099,0.9350,9.30693,0.7271,8.31683,6.324588,11.8812,1.142699,4.55446,8.61386,8.613861
9,HPE,15.71,N/A,3.08,6.33663,0.5162,6.13861,0.3664,2.77228,7.567494,16.8317,2.045866,11.0891,8.63366,8.633663


#Determining the Number of Shares to Buy


In [35]:
portfolio_size_case3 = input("Enter the value of your portfolio in Dollars:")

try:
    val = float(portfolio_size_case3)
except ValueError:
    print("Invalid Portfolio Size:")
    portfolio_size_case3 = input("Enter the value of your portfolio:")

Enter the value of your portfolio in Dollars:10000000


In [36]:
position_size_case3 = float(portfolio_size_case3) / len(val_strat_data.index)
for i in range(0, len(val_strat_data['Ticker'])-1):
    val_strat_data.loc[i, 'Number of Shares to Buy'] = math.floor(position_size_case3 / val_strat_data['Price'][i])
val_strat_data

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,Enterprise Value to EBITDA,EV/EBITDA Percentile,Enterprise Value to Gross Profit,EV/GP Percentile,RV Score,Robust Value Score
0,UNM,24.45,8179,6.23,9.70297,0.4428,5.74257,0.3755,3.16832,2.887049,3.16832,0.352408,0.792079,4.51485,4.514851
1,AAL,18.41,10863,-3.71,5.34653,-1.5900,5.54455,0.4877,4.9505,-7.310569,2.37624,1.455452,6.33663,4.91089,4.910891
2,HPQ,37.58,5321,3.60,7.0297,-12.6600,3.56436,0.3295,1.78218,3.908813,4.75248,1.680340,8.51485,5.12871,5.128713
3,PRU,111.47,1794,5.88,9.0099,0.6595,6.53465,0.5712,5.54455,4.461466,5.94059,0.589168,1.18812,5.64356,5.643564
4,BEN,33.25,6015,4.60,7.72277,0.7531,7.72277,0.9956,12.6733,3.003482,3.56436,0.817845,1.9802,6.73267,6.732673
5,BA,210.53,949,-14.54,4.15842,-8.2300,4.75248,1.9000,30.7921,-32.481961,0.594059,-1320.185523,0.19802,8.09901,8.099010
6,KSS,49.76,4019,7.95,12.6733,1.4100,16.8317,0.3721,2.9703,3.656899,3.9604,1.159481,4.9505,8.27723,8.277228
7,AIG,56.10,3565,8.91,13.8614,0.7277,7.32673,0.9860,12.0792,4.387631,5.74257,0.979202,3.16832,8.43564,8.435644
8,VIAC,29.55,6768,5.88,9.0099,0.9350,9.30693,0.7271,8.31683,6.324588,11.8812,1.142699,4.55446,8.61386,8.613861
9,HPE,15.71,12730,3.08,6.33663,0.5162,6.13861,0.3664,2.77228,7.567494,16.8317,2.045866,11.0891,8.63366,8.633663


#Formatting Our Excel Output

In [ ]:
val_strat_data.to_excel("Value_Investment_Watchlist.xlsx")
print('DataFrame is written to Excel File successfully.')
from google.colab import files
files.download('Value_Investment_Watchlist.xlsx')